In [ ]:
import json
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

## import company data

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://fortune.com/company/liberty-oilfield-services/fortune500/")
count = 0
fulldict = {}
while count < 1200:
    if count > 0:
        try:
            button = driver.find_element_by_class_name('companySinglePagination__prev--3QWI8')
            button.click()
        except:
            break
    try:
        cdict = {}
        cdict['Company Tag'] = driver.current_url.split('/')[-3]
        html = driver.page_source
        soup = BeautifulSoup(html)
        companysoup = soup.find("div", {"class": "info__wrapper--1CxpW"})
        for d in companysoup.find_all('div'):
            if len(d) > 1:
                dsplit = d.find_all('div')
                asplit = d.find_all('a')
                if len(dsplit) == 2:
                    cdict[dsplit[0].text] = dsplit[1].text
                elif len(asplit) == 1:
                    cdict[dsplit[0].text] = asplit[0].text
                else:
                    pass
        rank = int(soup.find("div", {"class": "companyTitle__rank--2SYbW"}).text.split('K')[-1].replace(' ', ''))
        cdict['Rank'] = rank
        companyname = soup.find("h1", {"class": "companyTitle__title--3Bdrv"}).find('a').find('div')
        fulldict[companyname.text] = cdict
    except:
        pass
    count += 1
driver.close()
driver.quit()

In [ ]:
with open('../data/f500_ranking_2020.json', 'w') as outfile:
    json.dump(fulldict, outfile)

## import company sectors

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://fortune.com/fortune500/2020/search/")

html = driver.page_source
soup = BeautifulSoup(html)

sectorsoup = soup.find('select', {'class': 'input__inputElement--1OLBW'})

sectors = []
for s in sectorsoup.find_all('option'):
    if s.text != '':
        sectors.append(s.text)

urlbuildups = {
    ',': '%2C',
    ' ': '%20',
    '&': '%26'
}

baseurl = 'https://fortune.com/fortune500/2020/search/?sector={}'
fulldict = {}
for e in sectors:
    e_clean = e
    for k, v in urlbuildups.items():
        e = e.replace(k, v)
    formattedurl = baseurl.format(e)
    
    driver.get(formattedurl)
    
    time.sleep(2)
    
    dropdown = driver.find_element_by_class_name('select-wrap')
    options = dropdown.find_elements_by_tag_name("option")
    options[-1].click()
    
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    subdict = {}
    for c in soup.find_all('div', {"class": "rt-tr-group"}):
        company = c.find_all('div')
        subdict[company[0].find_all('div')[1].text] = company[0].find_all('div')[2].text
    fulldict[e_clean] = subdict
driver.close()
driver.quit()

In [ ]:
with open('../data/f500_company_sectors.json', 'w') as outfile:
    json.dump(fulldict, outfile)